In [1]:
!pip uninstall fintech_utils -y

Found existing installation: fintech_utils 0.1.1
Uninstalling fintech_utils-0.1.1:
  Successfully uninstalled fintech_utils-0.1.1


In [2]:
!pip install git+https://github.com/SEANPNEX/fintech_utils.git

  Cloning https://github.com/SEANPNEX/fintech_utils.git to /private/var/folders/s2/6ffr7pmd1tv4rnht_h6c6hrr0000gn/T/pip-req-build-d4yakk6h
  Running command git clone --filter=blob:none --quiet https://github.com/SEANPNEX/fintech_utils.git /private/var/folders/s2/6ffr7pmd1tv4rnht_h6c6hrr0000gn/T/pip-req-build-d4yakk6h
  Resolved https://github.com/SEANPNEX/fintech_utils.git to commit 6af9bab9c2cc9846f1ea30cfcb454de74fa6b7a9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for fintech_utils: filename=fintech_utils-0.1.1-py3-none-any.whl size=17657 sha256=2e5b2204d3e531cb9103b4b428b7593dcec0a21c5a42f1d22509b0a0bfd06f70
  Stored in directory: /private/var/folders/s2/6ffr7pmd1tv4rnht_h6c6hrr0000gn/T/pip-ephem-wheel-cache-pp0yi7g8/wheels/ee/8c/ab/561d621c2e8083c2f8224aede161ed3dc49cfbf0fa06623e3f
Successfully built fintech_utils


In [3]:
import fintech_utils as ftu
from fintech_utils.findata.options import bsm_greeks
from fintech_utils.findata.options import binomial_greeks
from fintech_utils.findata.options import bt_american_discrete_div
import pandas as pd
import numpy as np

# 12.1

In [2]:
df = pd.read_csv("test12_1.csv")
df.set_index("ID", inplace=True)
df

,Option Type,Underlying,Strike,DaysToMaturity,DayPerYear,RiskFreeRate,DividendRate,ImpliedVol
ID,,,,,,,,
1.0,Call,100.0,100.0,50.0,365.0,0.045,0.00,0.2
2.0,Put,100.0,100.0,50.0,365.0,0.045,0.00,0.2
3.0,Call,100.0,90.0,400.0,365.0,0.045,0.02,0.4
4.0,Put,100.0,110.0,400.0,365.0,0.045,0.02,0.4
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
greek = {}
for i, row in df.iterrows():
    if pd.isna(i):
        continue
    S = row['Underlying']
    X = row['Strike']
    T = row["DaysToMaturity"] / row['DayPerYear']
    r = row['RiskFreeRate']
    q = row['DividendRate']
    sigma = row['ImpliedVol']
    option_type = row['Option Type'].lower()
    greeks = bsm_greeks(S, X, T, r, sigma, option_type=option_type, q=q)
    value = ftu.findata.options.bsm(S, X, T, r, sigma, option_type=option_type, q=q)
    val = {
        "Value": value,
        "Delta": greeks.delta(),
        "Gamma": greeks.gamma(),
        "Vega": greeks.vega(),
        "Rho": greeks.rho(),
        "Theta": greeks.theta(),
    }
    greek.update({i: val})
greek_df = pd.DataFrame.from_dict(greek, orient='index')
greek_df

,Value,Delta,Gamma,Vega,Rho,Theta
1.0,3.260824,0.547872,0.053506,14.659079,7.058414,-13.019817
2.0,2.646281,-0.452128,0.053506,14.659079,-6.556032,-8.547471
3.0,22.043329,0.685508,0.008115,35.571438,50.967099,-7.213608
4.0,20.449083,-0.470751,0.009310,40.812329,-73.999110,-5.351164


In [4]:
ans = pd.read_csv("testout12_1.csv")
ans

,ID,Value,Delta,Gamma,Vega,Rho,Theta
0,1,3.260824,0.547872,0.053506,14.659079,7.058414,-13.019817
1,2,2.646281,-0.452128,0.053506,14.659079,-6.556032,-8.547471
2,3,22.043329,0.685508,0.008115,35.571438,50.967099,-7.213608
3,4,20.449083,-0.470751,0.009310,40.812329,-73.999110,-5.351164


# 12.2

In [5]:
df = pd.read_csv("test12_1.csv")
df.set_index("ID", inplace=True)
df

,Option Type,Underlying,Strike,DaysToMaturity,DayPerYear,RiskFreeRate,DividendRate,ImpliedVol
ID,,,,,,,,
1.0,Call,100.0,100.0,50.0,365.0,0.045,0.00,0.2
2.0,Put,100.0,100.0,50.0,365.0,0.045,0.00,0.2
3.0,Call,100.0,90.0,400.0,365.0,0.045,0.02,0.4
4.0,Put,100.0,110.0,400.0,365.0,0.045,0.02,0.4
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
greek = {}
for i, row in df.iterrows():
    if pd.isna(i):
        continue
    S = row['Underlying']
    X = row['Strike']
    T = row["DaysToMaturity"] / row['DayPerYear']
    r = row['RiskFreeRate']
    q = row['DividendRate']
    sigma = row['ImpliedVol']
    option_type = row['Option Type'].lower()
    greeks = binomial_greeks(S, X, T, r, sigma, option_type=option_type, q=q, american=True)
    val = {
        "Value": greeks.price(),
        "Delta": greeks.delta(),
        "Gamma": greeks.gamma(),
        "Vega": greeks.vega(),
        "Rho": greeks.rho(),
        "Theta": greeks.theta(),
    }
    greek.update({i: val})
greek_df = pd.DataFrame.from_dict(greek, orient='index')
greek_df
    

,Value,Delta,Gamma,Vega,Rho,Theta
1.0,3.260085,0.547861,0.053546,14.655415,7.058353,13.027921
2.0,2.693228,-0.463231,0.055776,14.614871,-5.260221,8.949423
3.0,22.045853,0.685948,0.008144,35.914758,50.213594,7.239064
4.0,21.015110,-0.491737,0.010153,40.760725,-53.668631,5.948256


In [7]:
ans = pd.read_csv("testout12_2.csv")
ans.set_index("ID", inplace=True)
ans

,Value,Delta,Gamma,Vega,Rho,Theta
ID,,,,,,
1,3.259347,0.547849,0.055326,14.651751,-0.446486,13.014427
2,2.692775,-0.470676,0.057210,14.611961,-0.253731,8.939710
3,22.050266,0.676370,0.010698,35.904756,-22.898610,7.280144
4,21.019233,-0.489952,0.002654,40.759573,-14.102684,5.956511


# 12.3

In [4]:
df = pd.read_csv("test12_3.csv")
df

,ID,Option Type,Underlying,Strike,DaysToMaturity,DayPerYear,RiskFreeRate,ImpliedVol,DividendDates,DividendAmts
0,1,Call,100,100,250,365,0.045,0.4,"75,150",".01,.01"
1,2,Put,100,100,250,365,0.045,0.4,"75,150",".01,.01"


In [12]:
prices = {}
for i, row in df.iterrows():
    S = row['Underlying']
    X = row['Strike']
    T = row["DaysToMaturity"] / row['DayPerYear']
    r = row['RiskFreeRate']
    sigma = row['ImpliedVol']
    div_amts = [0.01, 0.01]
    div_time_steps = [75, 150]
    option_type = row['Option Type'].lower() == "call"
    am_option = bt_american_discrete_div(option_type, S, X, T, r, div_amts, div_time_steps, sigma, 500)
    prices.update({i: {"Value": am_option}})
    


In [14]:
pd.DataFrame.from_dict(prices, orient='index')

,Value
0,14.502728
1,11.780433


In [9]:
ans = pd.read_csv("testout12_3.csv")
ans

,ID,Value
0,1,14.503386
1,2,11.779632
